In [1]:
import numpy as np
import pandas as pd
import pickle
import os

from skimage.transform import resize

from extract_manual_features import extract_features

In [2]:
DIM = 64

In [3]:
if os.path.isfile('../data/X_MFE.pickle'):
    print('wafer map handcrafted features file exists')
else:
    print('wafer map handcrafted features file does not exist')
    print('dataframe preprocessing...')
    # Load wafer maps with labels
    df = pd.read_pickle('../data/LSWMD.pkl')
    df = df.drop(['waferIndex', 'trianTestLabel', 'lotName'], axis=1)
    df['failureNum']=df.failureType
    mapping_type={'Center':0,'Donut':1,'Edge-Loc':2,'Edge-Ring':3,'Loc':4,'Random':5,'Scratch':6,'Near-full':7,'none':8}
    df=df.replace({'failureNum':mapping_type})
    df_withlabel = df[(df['failureNum']>=0)]
    df_withlabel = df_withlabel.drop(df_withlabel[df_withlabel['dieSize']<100].index.tolist()).reset_index()
    
    X_mfe = extract_features(df_withlabel)
    y = np.array(df_withlabel['failureNum']).astype(np.int)
    
    # Save preprocessed data as pickle files
    with open('../data/X_MFE.pickle', 'wb') as f:
        pickle.dump(X_mfe, f, protocol=4)
    with open('../data/y.pickle', 'wb') as f:
        pickle.dump(y, f, protocol=4)

wafer map handcrafted features file does not exist
dataframe preprocessing...


C:\Users\huisleee\anaconda3\lib\site-packages\pandas\core\array_algos\replace.py:84: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)


density  0.3691292603810628 min


C:\Users\huisleee\anaconda3\lib\site-packages\skimage\transform\radon_transform.py:83: UserWarning: Radon transform: image must be zero outside the reconstruction circle
  warn('Radon transform: image must be zero outside the '


radon  21.885061101118723 min
geometry  5.9246962388356526 min
total 28.287 min


In [4]:
if os.path.isfile('../data/X_CNN'+str(DIM)+'.pickle'):
    print('wafer map image file exists')
else:
    print('wafer map image file does not exist')
    print('dataframe preprocessing...')
    # Load original data
    df = pd.read_pickle("../data/LSWMD.pkl")

    # Load wafer maps with labels
    df = df.drop(['waferIndex', 'trianTestLabel', 'lotName'], axis=1)
    df['failureNum']=df.failureType
    mapping_type={'Center':0,'Donut':1,'Edge-Loc':2,'Edge-Ring':3,'Loc':4,'Random':5,'Scratch':6,'Near-full':7,'none':8}
    df=df.replace({'failureNum':mapping_type})
    df_withlabel = df[(df['failureNum']>=0)]

    # Remove abnormal wafer maps with less than 100 dies
    df_withlabel = df_withlabel.drop(df_withlabel[df_withlabel['dieSize']<100].index.tolist()).reset_index()

    # Binarize and resize wafer maps
    X = df_withlabel.waferMap
    X_binary = [np.where(x<=1,0,1) for x in X]
    X_resize = np.array([resize(x,(DIM,DIM), preserve_range=True, anti_aliasing=False) for x in X_binary])
    X_resize = X_resize.reshape(-1,DIM,DIM,1).astype(np.float16)
    y = np.array(df_withlabel['failureNum']).astype(np.int)
    
    # Save preprocessed data as pickle files
    with open('../data/X_CNN_'+str(DIM)+'.pickle', 'wb') as f:
        pickle.dump(X_resize, f, protocol=4)

wafer map image file does not exist
dataframe preprocessing...


C:\Users\huisleee\anaconda3\lib\site-packages\pandas\core\array_algos\replace.py:84: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)
